**RTS Labs Coding Challenge - Data Engineering**

XYZ Travel website allows users to search and book flights, hotels, and cars. They want to
build a data warehouse/lake to gain customer intelligence. Website allows users to perform
below actions:

**Search**

The user searches according to the following attributes:

  **Flights**
-   Departure and arrival airport
-   Departure and arrival time
-   Number of passengers

  **Hotels**
-   Location
-   Number of rooms

  **Cars**
-   Model
-   Number of passengers

**Booking**

  The user can book any of the above items. They don't need to register to complete the
booking.
  Registered users can view their past reservations and also their previous five searches.

**Questions**

1. Please provide a database model for DWH
2. Please provide SQL to get: Hotels booked in 2021 and not in 2022
3. Please provide SQL to get: Amount spent on booking by registered users vs
non-registered users
4. Please provide SQL to get: Top 10 searches for each year
5. Please provide SQL to get: Percentage growth in booking by year and business lines
(Flight, Hotel, Car)

**1. Please provide a database model for DWH -- Redshift**

Amazon Redshift is a great choice for storing and analyzing large amounts of data because it works fast, can grow with your needs, is affordable, and connects easily with other AWS tools. It’s designed to handle big datasets and make sense of the information quickly and efficiently.

**Create Tables**

Fact Tables:
- `SearchFact`
- `BookingFact`

Dimension Tables:
- `UserDimension`
- `DateDimension`
- `BusinessLineDimension`


In [0]:
%sql
CREATE TABLE IF NOT EXISTS BookingFact (
    BookingID INT PRIMARY KEY,
    UserID INT,                               
    BookingDate DATE NOT NULL,              
    BusinessLine VARCHAR(50) NOT NULL,
    Amount DECIMAL(10, 2),        
    SearchID INT
);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS SearchFact (
    SearchID INT PRIMARY KEY,
    UserID INT,
    SearchDate DATE NOT NULL,
    BusinessLine VARCHAR(50) NOT NULL,
    SearchDetails STRING
);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS UserDimension (
    UserID BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    IsRegistered BOOLEAN NOT NULL
);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS DateDimension (
    DateKey DATE PRIMARY KEY,
    Year INT NOT NULL,
    Month INT NOT NULL,
    Day INT NOT NULL,
    DayOfWeek VARCHAR(10)
);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS BusinessLineDimension (
    BusinessLineID BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    BusinessLineName VARCHAR(50) NOT NULL
);


In [0]:
%sql

-- INSERT into UserDimension Table
INSERT INTO UserDimension (IsRegistered)
VALUES
(TRUE),
(FALSE),
(TRUE),
(FALSE);

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
-- INSERT into DateDimension Table
INSERT INTO DateDimension (DateKey, Year, Month, Day, DayOfWeek)
VALUES
('2021-01-01', 2021, 1, 1, 'Friday'),
('2021-06-15', 2021, 6, 15, 'Tuesday'),
('2022-07-20', 2022, 7, 20, 'Wednesday'),
('2022-08-05', 2022, 8, 5, 'Friday');


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
-- INSERT into BusinessLineDimension Table
INSERT INTO BusinessLineDimension (BusinessLineName)
VALUES
('Flight'),
('Hotel'),
('Car');

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
-- INSERT INTO SearchFact Table
INSERT INTO SearchFact (SearchID, UserID, SearchDate, BusinessLine, SearchDetails)
VALUES
(1, 1, '2021-06-15', 'Flight', '{"Departure": "NYC", "Arrival": "LAX"}'),
(2, 2, '2021-06-15', 'Hotel', '{"Location": "Paris", "Rooms": 2}'),
(3, 3, '2022-07-20', 'Car', '{"Model": "SUV", "Passengers": 4}'),
(4, NULL, '2022-08-05', 'Flight', '{"Departure": "LON", "Arrival": "BER"}');


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
-- INSERT INTO BookingFact Table
INSERT INTO BookingFact (BookingID, UserID, BookingDate, BusinessLine, Amount, SearchID)
VALUES
(1, 1, '2021-06-15', 'Hotel', 200.00, 2),
(2, 2, '2021-06-15', 'Flight', 500.00, 1),
(3, 3, '2022-07-20', 'Car', 150.00, 3),
(4, NULL, '2022-08-05', 'Flight', 300.00, 4);

num_affected_rows,num_inserted_rows
4,4


**Question 2 - 5**

In [0]:
%sql
-- Please provide SQL to get: Hotels booked in 2021 and not in 2022
SELECT DISTINCT bf.BookingID
FROM BookingFact bf
JOIN DateDimension dd ON bf.BookingDate = dd.DateKey
JOIN BusinessLineDimension bld ON bf.BusinessLine = bld.BusinessLineName
WHERE bld.BusinessLineName = 'Hotel' AND dd.Year = 2021
  AND NOT EXISTS (
    SELECT 1
    FROM BookingFact bf2
    JOIN DateDimension dd2 ON bf2.BookingDate = dd2.DateKey
    WHERE bf2.BusinessLine = 'Hotel' AND dd2.Year = 2022
  );

BookingID
1


In [0]:
%sql
-- Amount spent on booking by registered users vs non-registered users
SELECT 
    ud.IsRegistered,
    SUM(bf.Amount) AS TotalAmount
FROM BookingFact bf
LEFT JOIN UserDimension ud ON bf.UserID = ud.UserID
GROUP BY ud.IsRegistered;


IsRegistered,TotalAmount
null,3000.00
true,3500.00
false,5000.00


In [0]:
%sql
-- Top 10 Searches for Each Year
WITH SearchCounts AS (
    SELECT 
        YEAR(dd.DateKey) AS Year,
        sf.SearchDetails AS SearchAttributes,
        COUNT(*) AS SearchCount
    FROM SearchFact sf
    JOIN DateDimension dd ON sf.SearchDate = dd.DateKey
    GROUP BY YEAR(dd.DateKey), sf.SearchDetails
)
SELECT Year, SearchAttributes, SearchCount
FROM (
    SELECT 
        Year,
        SearchAttributes,
        SearchCount,
        ROW_NUMBER() OVER (PARTITION BY Year ORDER BY SearchCount DESC) AS Rank
    FROM SearchCounts
) Ranked
WHERE Rank <= 10;

Year,SearchAttributes,SearchCount
2021,"{""Location"": ""Paris"", ""Rooms"": 2}",100
2021,"{""Departure"": ""NYC"", ""Arrival"": ""LAX""}",100
2022,"{""Departure"": ""LON"", ""Arrival"": ""BER""}",100
2022,"{""Model"": ""SUV"", ""Passengers"": 4}",100


In [0]:
%sql
-- Percentage growth in booking by year and business lines (Flight, Hotel, Car)
WITH YearlyBookings AS (
    SELECT 
        YEAR(dd.DateKey) AS Year,
        bld.BusinessLineName,
        COUNT(*) AS TotalBookings
    FROM BookingFact bf
    JOIN DateDimension dd ON bf.BookingDate = dd.DateKey
    JOIN BusinessLineDimension bld ON bf.BusinessLine = bld.BusinessLineName
    GROUP BY YEAR(dd.DateKey), bld.BusinessLineName
),
Growth AS (
    SELECT 
        Current.Year,
        Current.BusinessLineName,
        ((Current.TotalBookings - Previous.TotalBookings) * 100.0 / Previous.TotalBookings) AS GrowthPercentage
    FROM YearlyBookings Current
    LEFT JOIN YearlyBookings Previous 
        ON Current.BusinessLineName = Previous.BusinessLineName AND Current.Year = Previous.Year + 1
)
SELECT Year, BusinessLineName, COALESCE(GrowthPercentage, 0) AS GrowthPercentage
FROM Growth;


Year,BusinessLineName,GrowthPercentage
2021,Hotel,0E-14
2022,Car,0E-14
2022,Flight,0E-14
2021,Flight,0E-14


### **ETL Exercise**


**Problem Statement:** 
  Create an ETL pipeline to extract data from the “user” API, transform
  and store the data into the DWH data model. Add logic to the ETL pipeline to process and
  store 1000 users in DB.

API info:
- To extract data - https://random-data-api.com/api/v2/users
- API docs - https://random-data-api.com/documentation

Tools or Tech info: Please feel free to use any tools or technology

In [0]:
%sql
USE ml.default;


In [0]:
%sql
-- Users Table
CREATE TABLE IF NOT EXISTS Users (
    id BIGINT PRIMARY KEY,
    uid VARCHAR(255),
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    username VARCHAR(255),
    email VARCHAR(255),
    gender VARCHAR(50),
    phone_number VARCHAR(50),
    social_insurance_number VARCHAR(50),
    date_of_birth DATE
);


In [0]:
%sql
-- Employment Table
CREATE TABLE IF NOT EXISTS Employment (
    user_id INT REFERENCES Users(id),
    title VARCHAR(255),
    key_skill VARCHAR(255)
);


In [0]:
%sql
-- Address Table
CREATE TABLE IF NOT EXISTS Address (
    user_id INT REFERENCES Users(id),
    street_address VARCHAR(255),
    street_name VARCHAR(255),
    city VARCHAR(100),
    state VARCHAR(100),
    zip_code VARCHAR(50),
    country VARCHAR(100),
    lat FLOAT,
    lng FLOAT
);


In [0]:
%sql
-- CreditCard Table
CREATE TABLE IF NOT EXISTS CreditCard (
    user_id INT REFERENCES Users(id),
    cc_number VARCHAR(255)
);


In [0]:
%sql
-- Subscription Table
CREATE TABLE IF NOT EXISTS Subscription (
    user_id INT REFERENCES Users(id),
    plan VARCHAR(50),
    status VARCHAR(50),
    payment_method VARCHAR(50),
    term VARCHAR(50)
);


In [0]:
%python
import requests
from pyspark.sql import SparkSession

# API Configuration
API_URL = "https://random-data-api.com/api/v2/users"
BATCH_SIZE = 1000

# Initialize Spark Session
spark = SparkSession.builder.appName("Simplified_ETL_Pipeline").getOrCreate()

# Step 1: Fetch Data from API
def fetch_users(api_url, batch_size):
    users = []
    for _ in range(batch_size):
        response = requests.get(api_url)
        if response.status_code == 200:
            users.append(response.json())
    return users

# Step 2: Transform Data
def transform_data(raw_data):
    return {
        "Users": [
            {
                "id": user["id"],
                "uid": user["uid"],
                "first_name": user["first_name"],
                "last_name": user["last_name"],
                "username": user["username"],
                "email": user["email"],
                "gender": user["gender"],
                "phone_number": user["phone_number"],
                "social_insurance_number": user["social_insurance_number"],
                "date_of_birth": user["date_of_birth"]
            }
            for user in raw_data
        ],
        "Employment": [
            {
                "user_id": user["id"],
                "title": user["employment"]["title"],
                "key_skill": user["employment"]["key_skill"]
            }
            for user in raw_data
        ],
        "Address": [
            {
                "user_id": user["id"],
                "street_address": user["address"]["street_address"],
                "city": user["address"]["city"],
                "state": user["address"]["state"],
                "zip_code": user["address"]["zip_code"],
                "country": user["address"]["country"],
                "lat": user["address"]["coordinates"]["lat"],
                "lng": user["address"]["coordinates"]["lng"]
            }
            for user in raw_data
        ],
        "CreditCard": [
            {"user_id": user["id"], "cc_number": user["credit_card"]["cc_number"]}
            for user in raw_data
        ],
        "Subscription": [
            {
                "user_id": user["id"],
                "plan": user["subscription"]["plan"],
                "status": user["subscription"]["status"],
                "payment_method": user["subscription"]["payment_method"],
                "term": user["subscription"]["term"]
            }
            for user in raw_data
        ]
    }

# Step 3: Load Data into Delta Tables
def load_to_delta(data, table_name):
    spark.createDataFrame(data).write.format("delta").mode("overwrite").saveAsTable(table_name)

# Main ETL Process
def run_etl():
    print("Starting ETL Pipeline...")
    raw_users = fetch_users(API_URL, BATCH_SIZE)
    transformed_data = transform_data(raw_users)
    
    for table, data in transformed_data.items():
        load_to_delta(data, f"default.{table}")

    print("ETL Pipeline completed successfully!")

# Execute ETL
run_etl()

Starting ETL Pipeline...
ETL Pipeline completed successfully!


In [0]:
%sql
SELECT * FROM Users LIMIT 10;

date_of_birth,email,first_name,gender,id,last_name,phone_number,social_insurance_number,uid,username
1981-12-14,cristobal.langworth@email.com,Cristobal,Female,1010,Langworth,+1-246 138.710.5848 x6879,256702820,0b79e6ff-cb87-49e5-9c0c-896d779e5008,cristobal.langworth
1986-10-06,jayson.herman@email.com,Jayson,Female,4510,Herman,+262 973-382-4314,475945127,a2e7d2f5-07cb-4dde-97a3-6c51a036ea96,jayson.herman
1989-05-06,maximo.kuphal@email.com,Maximo,Genderfluid,2783,Kuphal,+30 (746) 288-5733 x94656,966515983,b0efe891-400a-47f1-9c37-621f79c36d2e,maximo.kuphal
2000-04-26,fredrick.bechtelar@email.com,Fredrick,Polygender,8742,Bechtelar,+27 253.203.7846 x897,299835058,09f31b15-d7b3-41f4-b764-11945e9c5d5e,fredrick.bechtelar
1999-06-11,garland.ernser@email.com,Garland,Genderqueer,5739,Ernser,+964 1-108-741-1325 x5871,596082396,95aa2936-afe8-4154-a352-1795e21157c8,garland.ernser
1977-01-09,tommie.mueller@email.com,Tommie,Agender,7431,Mueller,+266 893-596-1501 x06104,769449240,1b3479fc-4423-443b-88ac-d602dcfcf53a,tommie.mueller
1982-12-22,josue.bernier@email.com,Josue,Female,8853,Bernier,+94 828-203-4829,473044972,e1e05389-9dfb-4458-93bd-3666234c87f9,josue.bernier
1991-11-04,raeann.schulist@email.com,Raeann,Genderqueer,1587,Schulist,+976 640-546-7618 x28679,968829911,40e386e4-eb4e-4bf0-af7c-0600b0eb0b39,raeann.schulist
1972-10-14,nannie.flatley@email.com,Nannie,Genderqueer,2574,Flatley,+504 760.845.8739 x9150,510779614,2d733421-bc4d-4443-b84f-4fa91b8fdaf7,nannie.flatley
1962-02-26,arlen.gleason@email.com,Arlen,Genderfluid,6361,Gleason,+54 1-933-551-5275 x4504,441198173,20160273-4b58-45ea-853b-30036d6dc8a9,arlen.gleason


In [0]:
%sql
SELECT u.first_name, u.last_name, e.title, e.key_skill, a.city, s.status, cc.cc_number
FROM Users u
JOIN Employment e ON u.id = e.user_id
JOIN Address a ON u.id = a.user_id
JOIN Subscription s ON u.id = s.user_id
JOIN CreditCard cc ON u.id = cc.user_id
LIMIT 10;


first_name,last_name,title,key_skill,city,status,cc_number
Isaiah,Swift,Sales Assistant,Confidence,North Eboni,Pending,4491-8838-7879-1406
Don,Lesch,Regional Banking Officer,Fast learner,Pourosview,Pending,6771-8944-0081-7128
Catherin,Kozey,Dynamic Sales Developer,Communication,South Mariano,Active,6771-8947-4773-3194
Brigitte,Schamberger,Investor Community-Services Executive,Organisation,Lelahfort,Pending,4606059260851
Stephany,Feest,Principal Education Consultant,Work under pressure,Moonburgh,Blocked,4868031796242
Nannie,Flatley,Central Accounting Designer,Leadership,Martyfort,Idle,6771-8919-2599-6478
Arlen,Gleason,Community-Services Engineer,Work under pressure,Lake Marcus,Pending,5121-0051-3307-4684
Reginald,Lehner,District Retail Designer,Technical savvy,West Leonie,Idle,4886192539078
Mitsuko,Johnston,Administration Associate,Fast learner,Celiaview,Blocked,5535-6153-0756-5331
Garland,Ernser,Principal Director,Fast learner,Jesusfurt,Pending,5237-3069-2310-1856
